<div style='float: right'><img src='pic/bridge.png'/></div>
## <div id='bridge' />橋をかけろ

In [ ]:
import numpy as np
from itertools import product
from pulp import *
from unionfind import unionfind
from ortoolpy import addvar, addvars, addbinvar, addbinvars
data = """\
.1.5.3..3
....1..2.
4..8.2..3
.2..2..2.
3..2.1..3
.6..5..4.
3..1.3..1
.3..1....
3..3.6.4.""".split()
nw, nh = len(data[0]), len(data)
r2 = range(2)
rhw = list(product(range(nh), range(nw)))

### 問題
* 島同士を線(橋)で結び、すべての数字が線でつながっているようにします
* 線は他の線と交差したり数字を横切ったりしてはいけません
* 線は水平方向か垂直方向に引かれます
* どの数の間にも2本までしか線は引けません
* 数字はその数字から引かれる線の数を表します

### 変数
* v：0:h1, 1:h2, 2:v1, 3:v2 (1)
* a：0:h, 1:v ∈{0,1,2} (2)

### 制約
* aをvで表します (3)
* 丸内が数字なら、周りの合計に等しくかつvは全て0 (4)
* h2==1ならh1==1であること。vも同じ (5)
* h1とv1は同時に成り立ちません (6)
* 数字以外では線が続き、端では線ははみ出ないこと (7)
* 全ての数字がつながること (8)

In [ ]:
m = LpProblem()
v = addbinvars(nh, nw, 4) # 0:h1, 1:h2, 2:v1, 3:v2 (1)
a = addvars(nh, nw, 2) # 0:h, 1:v (2)
def dirs(i, j):
    return [a[i + x + y - 1][j + x - y][1 - x ^ y] for y in r2 for x in r2
        if 0 <= i + x + y - 1 < nh and 0 <= j + x - y < nw]
for i, j in rhw:
    m += a[i][j][0] == lpSum(v[i][j][:2]) # (3)
    m += a[i][j][1] == lpSum(v[i][j][2:]) # (3)
    if data[i][j].isdigit():
        f = i*nh + j
        m += lpSum(dirs(i, j)) == int(data[i][j]) # (4)
        for k in range(4):
            m += v[i][j][k] == 0 # (4)
    else:
        m += v[i][j][0] >= v[i][j][1] # (5)
        m += v[i][j][2] >= v[i][j][3] # (5)
        m += lpSum(v[i][j][0:3:2]) <= 1 # (6)
        if i < nh-1 and not data[i+1][j].isdigit():
            m += a[i][j][1] == a[i+1][j][1] # (7)
        if j < nw-1 and not data[i][j+1].isdigit():
            m += a[i][j][0] == a[i][j+1][0] # (7)
        if 1 == 0 or 1 == nh-1:
            m += a[i][j][1] == 0 # (7)
        if j == 0 or j == nw-1:
            m += a[i][j][0] == 0 # (7)
while True:
    %time m.solve()
    if m.status == 1:
        break
    u = unionfind(nh * nw)
    e = []
    for j in rh:
        for i in rw:
            if value(a[i][j][0]):
                u.unite(i*nh + j, i*nh + j-1)
                u.unite(i*nh + j-1, i*nh + j+1)
            elif value(a[i][j][1]):
                u.unite(i*nh + j, i*nh-nh + j)
                u.unite(i*nh-nh + j, i*nh+nh + j)
            else:
                e.append(lpSum(a[i][j]))
    if all([u.issame(f, i*nh + j) for i, j in rhw if data[i][j].isdigit()]):
        break
    m += lpSum(e) >= 1 # (8)
for i in range(nh):
    for j in range(nw):
        h = int(sum(value(v[i][j][k]) * (3 if k == 2 else 1) for k in range(4)))
        print(data[i][j] if data[i][j] != '.' else ' -=|H'[h], end=' ')
    print()